# Proyecto final. Datamining.
### Análisis tweets UEFA Champions League Final 2018

### Participantes:

Gonzalo de las Heras de Matías - Jorge de la Fuente Tagarro - Alejandro Amarillas Cámara - Sergio Sampio Balmaseda.

### Notebook (1/4). Preprocesamiento y transformación del dataset.

### Objetivo del notebook:

Este notebook se centra en preprocesar el dataset, limpiando las columnas existentes y generando aquellas nuevas que sean necesarias.<br><br>

![title](Images/background.jpg)

#### Librerias

In [2]:
import pandas as pd
import numpy as np
import string
import math
import re
import nltk
import sys
import goslate
from matplotlib import *
from pylab import *
from datetime import datetime, timedelta
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from google.oauth2 import service_account
from google.cloud import translate
from sklearn.feature_extraction.text import TfidfVectorizer 

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('omw')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gonzalo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/gonzalo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/gonzalo/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package omw to /Users/gonzalo/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

#### Funciones

In [4]:
# helper function to clean tweets
def ProcesarTweet(Tweet):
    # Remove HTML special entities (e.g. &amp;)
    Tweet = re.sub(r"\&\w*;", "", Tweet)
    #Convert @username to AT_USER
    Tweet = re.sub("@[^\s]+","",Tweet)
    Tweet = re.sub("\[[^\s]+\]","",Tweet)
    # Remove tickers
    Tweet = re.sub(r"\$\w*", "", Tweet)
    # To lowercase
    Tweet = Tweet.lower()
    # Remove hyperlinks
    Tweet = re.sub(r"https?:\/\/.*\/\w*", "", Tweet)
    Tweet = re.sub(r"https...", "", Tweet)
    
    Tweet = re.sub(r"á", "a", Tweet)
    Tweet = re.sub(r"é", "e", Tweet)
    Tweet = re.sub(r"í", "i", Tweet)
    Tweet = re.sub(r"ó", "o", Tweet)
    Tweet = re.sub(r"ú", "u", Tweet)
    # Remove hashtags
    Tweet = re.sub(r"#\w*", "", Tweet)
    # Remove Punctuation and split "s, "t, "ve with a space for filter
    Tweet = re.sub(r"[" + string.punctuation.replace("@", "") + "]+", " ", Tweet)
    # Remove words with 2 or fewer letters
    Tweet = re.sub(r"\b\w{1,2}\b", "", Tweet)
    # Remove whitespace (including new line characters)
    Tweet = re.sub(r"\s\s+", " ", Tweet)
    # Remove single space remaining at the front of the tweet.
    Tweet = Tweet.lstrip(" ") 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    Tweet = "".join(c for c in Tweet if c <= "\uFFFF") 
    return Tweet

In [5]:
def text_process(raw_text):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in list(raw_text) if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = "".join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords.words("english")]

In [6]:
def CodificarPalabra(Palabra):
    return Palabra.encode("ascii", errors="ignore").decode()

## 1. Carga de datos

In [44]:
DIR_DATASET_JSON = "/Users/gonzalo/Documents/Datasets/tweets-ucl-final-2018/data.json"

In [45]:
if "json" not in globals():
    json = pd.read_json(DIR_DATASET_JSON, lines=True)

#### 1.3 Construcción dataset final

In [46]:
data = json.copy()
Dataset_Final = pd.DataFrame()

## 2. Limpieza de datos

Primera fase del proceso de KDD en el que limpiamos los datos

### 2.1 Eliminación de columnas

#### 2.1.1 Coordinates

In [47]:
if len(data[data['coordinates'] == None].index) == 0:
    print("La columna 'coordinates' está toda a NaN, borrando columna.")
    del data['coordinates']

La columna 'coordinates' está toda a NaN, borrando columna.


#### 2.1.2 Contributors

In [48]:
if len(data[data['contributors'] == data['contributors']].index) == 0:
    print("La columna 'contributors' está toda a NaN, borrando columna.")

del data['contributors']

La columna 'contributors' está toda a NaN, borrando columna.


#### 2.1.3 Withheld_in_countries

In [49]:
withheld_in_countries = len(data[data['withheld_in_countries'] == data['withheld_in_countries']].index)
if  withheld_in_countries== 0:
    print("La columna 'withheld_in_countries' está toda a NaN, borrando columna.")
    del data['withheld_in_countries']
else:
     print('Hay ', withheld_in_countries, "ocurrencias")

Hay  153 ocurrencias


Hay muy pocas ocurrencias que no sean nulas, eliminamos la columna entera al no ser de utilidad.

In [50]:
del data['withheld_in_countries']

#### 2.1.4 favorite_count

In [51]:
favorite_count = len(data[data['favorite_count'] != 0.0].index)
if  favorite_count == 0:
    print("La columna 'favorite_count' está toda a 0.0")
    del data['favorite_count']
else:
    print('Hay ', favorite_count, "ocurrencias")

Hay  24202 ocurrencias


#### 2.1.5 geo

In [52]:
geo_count = len(data[data['geo'] == None].index)
if geo_count == 0:
    print("La columna 'geo' está toda a NaN, borrando columna.")
    del data['geo']
else:
    print('Hay ', geo_count, "ocurrencias")

La columna 'geo' está toda a NaN, borrando columna.


#### 2.1.6 filter_level

In [53]:
filter_level_count = len(data['filter_level'].unique().tolist())
if filter_level_count == 1:
    print("La columna 'filter_level' es siempre igual.")
    del data['filter_level']
else:
    print('Hay ', filter_level_count, "valores distintos")

Hay  2 valores distintos


#### 2.1.6 place (se recogerá la información en la parte de usuarios)

Después de haber probado unos cuentos, nos quedamos con la localizacion de los usurios, casi siempre es la misma.

In [54]:
del data['place']

#### 2.1.8 Campos que no aportan información

In [55]:
del data['display_text_range']
del data['truncated']

### 2.2 Eliminación filas

#### 2.2.1 Eliminación filas nulas

In [56]:
data = data.drop(data[data['id'] != data['id']].id.index)
data = data.reset_index(drop=True)

Dataset_Final['id'] = data['id'].astype('int64')
del data['id']
del data['id_str']

### 2.3 Volcado al dataset final

#### 2.2.2 Conversión de fecha completa a solo tiempo (todos los tweets son del mismo día)

In [57]:
data.loc[:70,'created_at']

0    2018-05-26 13:18:30
1    2018-05-26 13:18:31
2    2018-05-26 13:18:31
3    2018-05-26 13:18:31
4    2018-05-26 13:18:31
5    2018-05-26 13:18:31
6    2018-05-26 13:18:31
7    2018-05-26 13:18:31
8    2018-05-26 13:18:31
9    2018-05-26 13:18:31
10   2018-05-26 13:18:31
11   2018-05-26 13:18:31
12   2018-05-26 13:18:31
13   2018-05-26 13:18:31
14   2018-05-26 13:18:31
15   2018-05-26 13:18:32
16   2018-05-26 13:18:32
17   2018-05-26 13:18:32
18   2018-05-26 13:18:32
19   2018-05-26 13:18:32
20   2018-05-26 13:18:32
21   2018-05-26 13:18:32
22   2018-05-26 13:18:32
23   2018-05-26 13:18:32
24   2018-05-26 13:18:33
25   2018-05-26 13:18:33
26   2018-05-26 13:18:33
27   2018-05-26 13:18:33
28   2018-05-26 13:18:33
29   2018-05-26 13:18:33
             ...        
41   2018-05-26 13:41:26
42   2018-05-26 13:41:26
43   2018-05-26 13:41:27
44   2018-05-26 13:41:27
45   2018-05-26 13:41:27
46   2018-05-26 13:41:27
47   2018-05-26 13:41:27
48   2018-05-26 13:41:28
49   2018-05-26 13:41:28


In [58]:
data['created_at'] += pd.DateOffset(hours=2) 
Dataset_Final['hora'] = data['created_at']
#del data['created_at']
#del data['timestamp_ms']

In [43]:
data.loc[:70,'created_at']

0    2018-05-26 15:18:30
1    2018-05-26 15:18:31
2    2018-05-26 15:18:31
3    2018-05-26 15:18:31
4    2018-05-26 15:18:31
5    2018-05-26 15:18:31
6    2018-05-26 15:18:31
7    2018-05-26 15:18:31
8    2018-05-26 15:18:31
9    2018-05-26 15:18:31
10   2018-05-26 15:18:31
11   2018-05-26 15:18:31
12   2018-05-26 15:18:31
13   2018-05-26 15:18:31
14   2018-05-26 15:18:31
15   2018-05-26 15:18:32
16   2018-05-26 15:18:32
17   2018-05-26 15:18:32
18   2018-05-26 15:18:32
19   2018-05-26 15:18:32
20   2018-05-26 15:18:32
21   2018-05-26 15:18:32
22   2018-05-26 15:18:32
23   2018-05-26 15:18:32
24   2018-05-26 15:18:33
25   2018-05-26 15:18:33
26   2018-05-26 15:18:33
27   2018-05-26 15:18:33
28   2018-05-26 15:18:33
29   2018-05-26 15:18:33
             ...        
41   2018-05-26 15:41:26
42   2018-05-26 15:41:26
43   2018-05-26 15:41:27
44   2018-05-26 15:41:27
45   2018-05-26 15:41:27
46   2018-05-26 15:41:27
47   2018-05-26 15:41:27
48   2018-05-26 15:41:28
49   2018-05-26 15:41:28


#### 2.2.3 Formateo de fuente (plataforma desde el que se lanza el tweet)

In [120]:
Dataset_Final['fuente'] = data['source'].replace('<a[^>]*>([^<]+)<\/a>', r'\1', regex=True)

Dataset_Final.loc[Dataset_Final['fuente'] == 'Twitter for Android', 'fuente'] = 'Android'
Dataset_Final.loc[Dataset_Final['fuente'] == 'Twitter for iPhone', 'fuente']  = 'iPhone'
Dataset_Final.loc[Dataset_Final['fuente'] == 'Twitter for iPad', 'fuente']  = 'iPad'
Dataset_Final.loc[Dataset_Final['fuente'] == 'Twitter Web Client', 'fuente']  = 'Twitter Web'
del data['source']

#### 2.2.4 Idioma

In [121]:
Dataset_Final['idioma'] = data['lang']
del data['lang']

#### 2.2.5 Id tweet respuesta

In [122]:
data['in_reply_to_status_id'].fillna(-1)
Dataset_Final['tweet_respuesta_id'] = data['in_reply_to_status_id']
del data['in_reply_to_status_id']
del data['in_reply_to_status_id_str']

#### 2.2.6 Id usuario tweet respuesta

In [123]:
data['in_reply_to_user_id'].fillna(-1)
Dataset_Final['tweet_respuesta_usuario_id'] = data['in_reply_to_user_id']
del data['in_reply_to_user_id']
del data['in_reply_to_user_id_str']

#### 2.2.7 Nombre usuario tweet respuesta

In [124]:
data['in_reply_to_screen_name'].fillna(-1)
Dataset_Final['tweet_respuesta_nombre_twitter'] = data['in_reply_to_screen_name']
del data['in_reply_to_screen_name']

#### 2.2.8 Información anidada

In [125]:
id_usuarios = []
name = [] 
screen_name = []
description = []
verified = []
location = []
followers_count = []
listed_count = []
favourites_count = []
created_at = []
usuario_lang = []
statuses_count = []
text = []
hashtag = []

user_mentions_screen_name = []
user_mentions_id = []
user_mentions_name = []

for index, row in data.iterrows():
    datos_usuario = row["user"]
    extended_tweet = row["extended_tweet"]
    entities = row["entities"] 
    
    statuses_count.append(datos_usuario["statuses_count"])
    location.append(datos_usuario["location"])
    id_usuarios.append(datos_usuario["id"])
    name.append(datos_usuario["name"])
    screen_name.append(datos_usuario["screen_name"])
    description.append(datos_usuario["description"])
    verified.append(datos_usuario["verified"])
    followers_count.append(datos_usuario["followers_count"])
    favourites_count.append(datos_usuario["favourites_count"])
    created_at.append(datos_usuario["created_at"])
    usuario_lang.append(datos_usuario["lang"])
    
    # Menciones en un tweet.
    user_mentions_screen_name_temp = []
    user_mentions_id_temp = []
    user_mentions_name_temp = []
    
    for user in entities["user_mentions"]:
        user_mentions_screen_name_temp.append(user["screen_name"])
        user_mentions_id_temp.append(user["id"])
        user_mentions_name_temp.append(user["name"])
        
    if len(user_mentions_screen_name_temp) > 0:
        user_mentions_screen_name.append(user_mentions_screen_name_temp)
    else:
        user_mentions_screen_name.append(-1)
        
    if len(user_mentions_id_temp) > 0:
        user_mentions_id.append(user_mentions_id_temp)
    else:
        user_mentions_id.append(-1)
        
    if len(user_mentions_name_temp) > 0:
        user_mentions_name.append(user_mentions_name_temp)
    else:
        user_mentions_name.append(-1)
    
    if extended_tweet is not "nan":
        text.append(row["text"])
        hashtags_temp = []
        for i in range(0, len(entities["hashtags"])):
            temp_hashtag = CodificarPalabra(entities["hashtags"][i]["text"].lower())
            if len(temp_hashtag) > 0:
                hashtags_temp.append(temp_hashtag)
        if len(hashtags_temp) > 0:
            hashtag.append(hashtags_temp)
        else:
            hashtag.append(-1)
    else:
        text.append(extended_tweet["full_text"])
        hashtags_temp = []
        for i in range(0, len(extended_tweet["entities"]["hashtags"])):
               hashtags_temp.append(extended_tweet["entities"]["hashtags"][i]["text"].lower())
        if len(hashtags_temp) > 0:
            hashtag.append(hashtags_temp)
        else:
            hashtag.append(-1)

In [126]:
Dataset_Final['usuario_id'] = id_usuarios
Dataset_Final['usuario_nombre'] = name
Dataset_Final['usuario_nombre_twitter'] = screen_name
Dataset_Final['usuario_localizacion'] = location
Dataset_Final['usuario_idioma'] = usuario_lang
Dataset_Final['usuario_verificado'] = verified
Dataset_Final['usuario_numero_seguidores'] = followers_count
Dataset_Final['usuario_numero_favoritos_hechos'] = favourites_count
Dataset_Final['usuario_numero_tweets'] = statuses_count
Dataset_Final['usuario_numero_creacion'] = created_at
Dataset_Final['hashtag'] = hashtag
Dataset_Final['texto_original'] = text
Dataset_Final['mencion_usuario_id'] = user_mentions_id
Dataset_Final['mencion_usuario_nombre'] = user_mentions_name
Dataset_Final['mencion_usuario_nombre_twitter'] = user_mentions_screen_name

del data['user']
del data['entities']

In [127]:
#Dataset_Final = Dataset_Final[:1000]
#Dataset_Final

In [128]:
Dataset_Final.fillna(-1, inplace=True)
Dataset_Final.tweet_respuesta_id = Dataset_Final.tweet_respuesta_id.astype(int)
Dataset_Final.tweet_respuesta_usuario_id = Dataset_Final.tweet_respuesta_usuario_id.astype(int)

## Transformación y generación

In [129]:
Dataset_Final['texto_limpio'] = Dataset_Final['texto_original'].apply(ProcesarTweet)
Dataset_Final['texto_traducido'] = "-1"

In [130]:
credentials = service_account.Credentials.from_service_account_file("/Users/gonzalo/Documents/Keys/google.json")
translate_client = translate.Client(credentials=credentials)

terminado = False
    
print("Hay", len(Dataset_Final), "filas")

for index, row in Dataset_Final.iterrows():
    if index % 100 == 0:
        print(index, "filas traducidas.")
        
    if row["idioma"] != "en" and row["texto_traducido"] == "-1":
        
        # The target language
        target = 'en'

        # Translates some text into Russian
        translation = translate_client.translate(row["texto_limpio"], target_language=target)
        Dataset_Final.loc[index, 'texto_traducido'] = translation['translatedText']
        
    else:
        
        Dataset_Final.loc[index, 'texto_traducido'] = row['texto_limpio']

Hay 330384 filas
0 filas traducidas.
100 filas traducidas.
200 filas traducidas.
300 filas traducidas.
400 filas traducidas.
500 filas traducidas.
600 filas traducidas.
700 filas traducidas.
800 filas traducidas.
900 filas traducidas.
1000 filas traducidas.
1100 filas traducidas.
1200 filas traducidas.
1300 filas traducidas.
1400 filas traducidas.
1500 filas traducidas.
1600 filas traducidas.
1700 filas traducidas.
1800 filas traducidas.
1900 filas traducidas.
2000 filas traducidas.
2100 filas traducidas.
2200 filas traducidas.
2300 filas traducidas.
2400 filas traducidas.
2500 filas traducidas.
2600 filas traducidas.
2700 filas traducidas.
2800 filas traducidas.
2900 filas traducidas.
3000 filas traducidas.
3100 filas traducidas.
3200 filas traducidas.
3300 filas traducidas.
3400 filas traducidas.
3500 filas traducidas.
3600 filas traducidas.
3700 filas traducidas.
3800 filas traducidas.
3900 filas traducidas.
4000 filas traducidas.
4100 filas traducidas.
4200 filas traducidas.
4300 f

34600 filas traducidas.
34700 filas traducidas.
34800 filas traducidas.
34900 filas traducidas.
35000 filas traducidas.
35100 filas traducidas.
35200 filas traducidas.
35300 filas traducidas.
35400 filas traducidas.
35500 filas traducidas.
35600 filas traducidas.
35700 filas traducidas.
35800 filas traducidas.
35900 filas traducidas.
36000 filas traducidas.
36100 filas traducidas.
36200 filas traducidas.
36300 filas traducidas.
36400 filas traducidas.
36500 filas traducidas.
36600 filas traducidas.
36700 filas traducidas.
36800 filas traducidas.
36900 filas traducidas.
37000 filas traducidas.
37100 filas traducidas.
37200 filas traducidas.
37300 filas traducidas.
37400 filas traducidas.
37500 filas traducidas.
37600 filas traducidas.
37700 filas traducidas.
37800 filas traducidas.
37900 filas traducidas.
38000 filas traducidas.
38100 filas traducidas.
38200 filas traducidas.
38300 filas traducidas.
38400 filas traducidas.
38500 filas traducidas.
38600 filas traducidas.
38700 filas trad

68800 filas traducidas.
68900 filas traducidas.
69000 filas traducidas.
69100 filas traducidas.
69200 filas traducidas.
69300 filas traducidas.
69400 filas traducidas.
69500 filas traducidas.
69600 filas traducidas.
69700 filas traducidas.
69800 filas traducidas.
69900 filas traducidas.
70000 filas traducidas.
70100 filas traducidas.
70200 filas traducidas.
70300 filas traducidas.
70400 filas traducidas.
70500 filas traducidas.
70600 filas traducidas.
70700 filas traducidas.
70800 filas traducidas.
70900 filas traducidas.
71000 filas traducidas.
71100 filas traducidas.
71200 filas traducidas.
71300 filas traducidas.
71400 filas traducidas.
71500 filas traducidas.
71600 filas traducidas.
71700 filas traducidas.
71800 filas traducidas.
71900 filas traducidas.
72000 filas traducidas.
72100 filas traducidas.
72200 filas traducidas.
72300 filas traducidas.
72400 filas traducidas.
72500 filas traducidas.
72600 filas traducidas.
72700 filas traducidas.
72800 filas traducidas.
72900 filas trad

102900 filas traducidas.
103000 filas traducidas.
103100 filas traducidas.
103200 filas traducidas.
103300 filas traducidas.
103400 filas traducidas.
103500 filas traducidas.
103600 filas traducidas.
103700 filas traducidas.
103800 filas traducidas.
103900 filas traducidas.
104000 filas traducidas.
104100 filas traducidas.
104200 filas traducidas.
104300 filas traducidas.
104400 filas traducidas.
104500 filas traducidas.
104600 filas traducidas.
104700 filas traducidas.
104800 filas traducidas.
104900 filas traducidas.
105000 filas traducidas.
105100 filas traducidas.
105200 filas traducidas.
105300 filas traducidas.
105400 filas traducidas.
105500 filas traducidas.
105600 filas traducidas.
105700 filas traducidas.
105800 filas traducidas.
105900 filas traducidas.
106000 filas traducidas.
106100 filas traducidas.
106200 filas traducidas.
106300 filas traducidas.
106400 filas traducidas.
106500 filas traducidas.
106600 filas traducidas.
106700 filas traducidas.
106800 filas traducidas.


135700 filas traducidas.
135800 filas traducidas.
135900 filas traducidas.
136000 filas traducidas.
136100 filas traducidas.
136200 filas traducidas.
136300 filas traducidas.
136400 filas traducidas.
136500 filas traducidas.
136600 filas traducidas.
136700 filas traducidas.
136800 filas traducidas.
136900 filas traducidas.
137000 filas traducidas.
137100 filas traducidas.
137200 filas traducidas.
137300 filas traducidas.
137400 filas traducidas.
137500 filas traducidas.
137600 filas traducidas.
137700 filas traducidas.
137800 filas traducidas.
137900 filas traducidas.
138000 filas traducidas.
138100 filas traducidas.
138200 filas traducidas.
138300 filas traducidas.
138400 filas traducidas.
138500 filas traducidas.
138600 filas traducidas.
138700 filas traducidas.
138800 filas traducidas.
138900 filas traducidas.
139000 filas traducidas.
139100 filas traducidas.
139200 filas traducidas.
139300 filas traducidas.
139400 filas traducidas.
139500 filas traducidas.
139600 filas traducidas.


168500 filas traducidas.
168600 filas traducidas.
168700 filas traducidas.
168800 filas traducidas.
168900 filas traducidas.
169000 filas traducidas.
169100 filas traducidas.
169200 filas traducidas.
169300 filas traducidas.
169400 filas traducidas.
169500 filas traducidas.
169600 filas traducidas.
169700 filas traducidas.
169800 filas traducidas.
169900 filas traducidas.
170000 filas traducidas.
170100 filas traducidas.
170200 filas traducidas.
170300 filas traducidas.
170400 filas traducidas.
170500 filas traducidas.
170600 filas traducidas.
170700 filas traducidas.
170800 filas traducidas.
170900 filas traducidas.
171000 filas traducidas.
171100 filas traducidas.
171200 filas traducidas.
171300 filas traducidas.
171400 filas traducidas.
171500 filas traducidas.
171600 filas traducidas.
171700 filas traducidas.
171800 filas traducidas.
171900 filas traducidas.
172000 filas traducidas.
172100 filas traducidas.
172200 filas traducidas.
172300 filas traducidas.
172400 filas traducidas.


201300 filas traducidas.
201400 filas traducidas.
201500 filas traducidas.
201600 filas traducidas.
201700 filas traducidas.
201800 filas traducidas.
201900 filas traducidas.
202000 filas traducidas.
202100 filas traducidas.
202200 filas traducidas.
202300 filas traducidas.
202400 filas traducidas.
202500 filas traducidas.
202600 filas traducidas.
202700 filas traducidas.
202800 filas traducidas.
202900 filas traducidas.
203000 filas traducidas.
203100 filas traducidas.
203200 filas traducidas.
203300 filas traducidas.
203400 filas traducidas.
203500 filas traducidas.
203600 filas traducidas.
203700 filas traducidas.
203800 filas traducidas.
203900 filas traducidas.
204000 filas traducidas.
204100 filas traducidas.
204200 filas traducidas.
204300 filas traducidas.
204400 filas traducidas.
204500 filas traducidas.
204600 filas traducidas.
204700 filas traducidas.
204800 filas traducidas.
204900 filas traducidas.
205000 filas traducidas.
205100 filas traducidas.
205200 filas traducidas.


234100 filas traducidas.
234200 filas traducidas.
234300 filas traducidas.
234400 filas traducidas.
234500 filas traducidas.
234600 filas traducidas.
234700 filas traducidas.
234800 filas traducidas.
234900 filas traducidas.
235000 filas traducidas.
235100 filas traducidas.
235200 filas traducidas.
235300 filas traducidas.
235400 filas traducidas.
235500 filas traducidas.
235600 filas traducidas.
235700 filas traducidas.
235800 filas traducidas.
235900 filas traducidas.
236000 filas traducidas.
236100 filas traducidas.
236200 filas traducidas.
236300 filas traducidas.
236400 filas traducidas.
236500 filas traducidas.
236600 filas traducidas.
236700 filas traducidas.
236800 filas traducidas.
236900 filas traducidas.
237000 filas traducidas.
237100 filas traducidas.
237200 filas traducidas.
237300 filas traducidas.
237400 filas traducidas.
237500 filas traducidas.
237600 filas traducidas.
237700 filas traducidas.
237800 filas traducidas.
237900 filas traducidas.
238000 filas traducidas.


266900 filas traducidas.
267000 filas traducidas.
267100 filas traducidas.
267200 filas traducidas.
267300 filas traducidas.
267400 filas traducidas.
267500 filas traducidas.
267600 filas traducidas.
267700 filas traducidas.
267800 filas traducidas.
267900 filas traducidas.
268000 filas traducidas.
268100 filas traducidas.
268200 filas traducidas.
268300 filas traducidas.
268400 filas traducidas.
268500 filas traducidas.
268600 filas traducidas.
268700 filas traducidas.
268800 filas traducidas.
268900 filas traducidas.
269000 filas traducidas.
269100 filas traducidas.
269200 filas traducidas.
269300 filas traducidas.
269400 filas traducidas.
269500 filas traducidas.
269600 filas traducidas.
269700 filas traducidas.
269800 filas traducidas.
269900 filas traducidas.
270000 filas traducidas.
270100 filas traducidas.
270200 filas traducidas.
270300 filas traducidas.
270400 filas traducidas.
270500 filas traducidas.
270600 filas traducidas.
270700 filas traducidas.
270800 filas traducidas.


299700 filas traducidas.
299800 filas traducidas.
299900 filas traducidas.
300000 filas traducidas.
300100 filas traducidas.
300200 filas traducidas.
300300 filas traducidas.
300400 filas traducidas.
300500 filas traducidas.
300600 filas traducidas.
300700 filas traducidas.
300800 filas traducidas.
300900 filas traducidas.
301000 filas traducidas.
301100 filas traducidas.
301200 filas traducidas.
301300 filas traducidas.
301400 filas traducidas.
301500 filas traducidas.
301600 filas traducidas.
301700 filas traducidas.
301800 filas traducidas.
301900 filas traducidas.
302000 filas traducidas.
302100 filas traducidas.
302200 filas traducidas.
302300 filas traducidas.
302400 filas traducidas.
302500 filas traducidas.
302600 filas traducidas.
302700 filas traducidas.
302800 filas traducidas.
302900 filas traducidas.
303000 filas traducidas.
303100 filas traducidas.
303200 filas traducidas.
303300 filas traducidas.
303400 filas traducidas.
303500 filas traducidas.
303600 filas traducidas.


In [131]:
vectorizer = TfidfVectorizer(max_df=0.5,max_features=10000,min_df=7,stop_words='english',use_idf=True)
X = vectorizer.fit_transform(Dataset_Final['texto_traducido'].str.upper())
sid = SentimentIntensityAnalyzer()
Dataset_Final['sentimiento_compound_polarity']=Dataset_Final["texto_traducido"].apply(lambda x:sid.polarity_scores(x)['compound'])
Dataset_Final['sentimiento_neutral']=Dataset_Final["texto_traducido"].apply(lambda x:sid.polarity_scores(x)['neu'])
Dataset_Final['sentimiento_negativo']=Dataset_Final["texto_traducido"].apply(lambda x:sid.polarity_scores(x)['neg'])
Dataset_Final['sentimiento_positivo']=Dataset_Final["texto_traducido"].apply(lambda x:sid.polarity_scores(x)['pos'])
Dataset_Final['sentimiento_tipo']= ""
Dataset_Final.loc[Dataset_Final['sentimiento_compound_polarity']>0,'sentimiento_tipo']='POSITIVE'
Dataset_Final.loc[Dataset_Final['sentimiento_compound_polarity']==0,'sentimiento_tipo']='NEUTRAL'
Dataset_Final.loc[Dataset_Final['sentimiento_compound_polarity']<0,'sentimiento_tipo']='NEGATIVE'

In [132]:
Dataset_Final['tokens'] = Dataset_Final['texto_traducido'].apply(text_process)

In [ ]:
Dataset_Final= Dataset_Final.reset_index(drop=True)
for index, row in Dataset_Final.iterrows():
    hashtags = []
    print(index)
    if row["hashtag"] != "-1" and row["hashtag"] != -1:
        for hastag in row["hashtag"]:
            if hastag != "_" and hastag != "__" and hastag != "___" and hastag != "uclfinal":
                hashtags.append(hastag)
    if len(hashtags) > 0:
        Dataset_Final.set_value(index, "hashtag", hashtags)
    else:
        Dataset_Final.loc[index, "hashtag"] = "-1"

In [133]:
Dataset_Final.to_json("xaca.json")

In [134]:
Dataset_Final.head()

,id,hora,fuente,idioma,tweet_respuesta_id,tweet_respuesta_usuario_id,tweet_respuesta_nombre_twitter,usuario_id,usuario_nombre,usuario_nombre_twitter,...,mencion_usuario_nombre,mencion_usuario_nombre_twitter,texto_limpio,texto_traducido,sentimiento_compound_polarity,sentimiento_neutral,sentimiento_negativo,sentimiento_positivo,sentimiento_tipo,tokens
0,1000365563376488448,15:18:30,Hootsuite,en,-1,-1,-1,2846595478,Real Madrid vs Liverpool Live stream,UCL_TV,...,-1,-1,match day real madrid liverpool free live stre...,match day real madrid liverpool free live stre...,0.5106,0.708,0.0,0.292,POSITIVE,"[match, day, real, madrid, liverpool, free, li..."
1,1000365564190048256,15:18:31,Android,in,-1,-1,-1,2917613580,Individu Merdeka ☮,Raditakia,...,[PanditFootball.com],[panditfootball],siapa yang punya peluang paling besar jadi jua...,who has the biggest chance of becoming 2018 ch...,0.8316,0.476,0.0,0.524,POSITIVE,"[biggest, chance, becoming, 2018, champions, c..."
2,1000365564320247808,15:18:31,Android,es,-1,-1,-1,946054253461852160,Roberto Esperón alonso,esperon_alonso,...,"[Madridismo Real™, Real Madrid C.F. ⚽️, Nissan...","[madridismoreaI_, realmadrid, Nissan_ESP]",llegado dia ⚽️ hacerlo real,come day ⚽️ make it real,0.0000,1.000,0.0,0.000,NEUTRAL,"[come, day, ⚽️, make, real]"
3,1000365564357832704,15:18:31,Madridista2,es,-1,-1,-1,962840767,R.Madrid™,RMadridIsco,...,[Madridismo Real™],[madridismoreaI_],para mañana navas carvajal nacho ramos marcelo...,for tomorrow navas carvajal nacho ramos marcel...,0.0000,1.000,0.0,0.000,NEUTRAL,"[tomorrow, navas, carvajal, nacho, ramos, marc..."
4,1000365564517388288,15:18:31,Android,en,-1,-1,-1,902735000445095938,Kwame-Nat,Kwame_Nat1,...,[ECG],[ECG_Unofficial],will like categorically state our sincere comm...,will like categorically state our sincere comm...,0.8555,0.490,0.0,0.510,POSITIVE,"[like, categorically, state, sincere, commitme..."


In [135]:
fg

NameError: name 'fg' is not defined

In [1]:
Dataset_Final

NameError: name 'Dataset_Final' is not defined

In [ ]:

Dataset_Final.head()

In [ ]:
# split sentences to get individual words
all_words = []
for line in Dataset_Final['tokens']: # try 'tokens'
    all_words.extend(line)
    
# create a word frequency dictionary
wordfreq = Counter(all_words)
# draw a Word Cloud with word frequencies
wordcloud = WordCloud(width=1600,
                      height=800,
                      max_words=200,
                      max_font_size=100,
                      relative_scaling=0.5,
                      colormap="ocean",
                      normalize_plurals=True).generate_from_frequencies(wordfreq)

plt.figure( figsize=(20,10), facecolor='k')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# split sentences to get individual words
all_words = []
for line in Dataset_Final[Dataset_Final['hashtag'] != -1]['hashtag']: # try 'tokens'
    all_words.extend(line)
    
# create a word frequency dictionary
wordfreq = Counter(all_words)

alice_mask = np.array(Image.open(path.join(d, "5.jpg")))

# draw a Word Cloud with word frequencies
wc = WordCloud(
                      max_words=50,
                      mask=alice_mask,
                      contour_color='steelblue').generate_from_frequencies(wordfreq)


# store to file
#wc.to_file(path.join(d, "alice.png"))
plt.figure(figsize=(17,14))
# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.show()

In [ ]:
Dataset_Final.loc[0].texto_limpio

In [ ]:
translation

In [ ]:
#!/usr/bin/env python
"""
Masked wordcloud
================

Using a mask you can generate wordclouds in arbitrary shapes.
"""

from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# Read the whole text.
text = open(path.join(d, '2.txt')).read()

# read the mask image
# taken from
# http://www.stencilry.org/stencils/movies/alice%20in%20wonderland/255fk.jpg
alice_mask = np.array(Image.open(path.join(d, "3.jpg")))

stopwords = set(STOPWORDS)
stopwords.add("said")

wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
               stopwords=stopwords, contour_color='steelblue')

# generate word cloud
wc.generate(text)

# store to file
wc.to_file(path.join(d, "alice.png"))

# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis("off")
plt.show()